# Homorphic operations over Decision Tree

## IRIS Data Set 
<p>This demo is using the well known IRIS data set.</p>
The data set consists of 50 samples from each of three species of Iris (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimeters. 

## Load IRIS dataset, split to train and test sets and run DecisionTreeClassifier

In [1]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.model_selection import train_test_split

iris = load_iris()

# split test set for prdictions
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, random_state=42)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

## Extract All the paths from the decision tree 

In [2]:
import copy
def get_path(tree, feature_names, clazz):
        left      = tree.children_left
        right     = tree.children_right
        threshold = tree.threshold
        value = tree.value
        features = tree.feature
        result = []
        
        def recurse(left, right, node, tree_path):
                if (threshold[node] != -2):
                        if left[node] != -1:
                                t1 = copy.deepcopy(tree_path)
                                t1.append( [ feature_names[ features[node] ] , "<=" , threshold[node] ])
                                recurse (left, right, left[node], t1)
                        if right[node] != -1:
                                t2 = copy.deepcopy(tree_path)
                                t2.append( [ feature_names[ features[node] ] , ">" , threshold[node] ])
                                recurse (left, right, right[node], t2)
                else:
                        v = value[node]
                        if np.max(v) == v.item(clazz):
                            #print ("path: " , tree_path, " ", str(v) )
                            result.append(tree_path)

        recurse(left, right,  0, [] )
        return result

## Extract all the paths for Vesicolor ( class: 2 )

In [3]:
#define feature name
feauters = ["sepal_length","sepal_width","petal_length","petal_width"]
path_tree = get_path(clf.tree_, feauters , 2 )
path_tree

[[['petal_width', '>', 0.800000011920929],
  ['petal_width', '<=', 1.75],
  ['petal_length', '<=', 4.950000047683716],
  ['petal_width', '>', 1.6500000357627869]],
 [['petal_width', '>', 0.800000011920929],
  ['petal_width', '<=', 1.75],
  ['petal_length', '>', 4.950000047683716],
  ['petal_width', '<=', 1.550000011920929]],
 [['petal_width', '>', 0.800000011920929],
  ['petal_width', '<=', 1.75],
  ['petal_length', '>', 4.950000047683716],
  ['petal_width', '>', 1.550000011920929],
  ['petal_length', '>', 5.450000047683716]],
 [['petal_width', '>', 0.800000011920929],
  ['petal_width', '>', 1.75],
  ['petal_length', '<=', 4.8500001430511475],
  ['sepal_length', '>', 5.950000047683716]],
 [['petal_width', '>', 0.800000011920929],
  ['petal_width', '>', 1.75],
  ['petal_length', '>', 4.8500001430511475]]]

## Make Prediction using DecisionTreeClassifier

In [4]:
y_pred = clf.predict(X_test)
y_pred

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0])

 ## Make Prediction by evauating the path list for Versicolor
   
 <b>This will be first the binary AND on the individual path, e.g</b>
    
 ```python   
  'petal_width', '>', 0.800000011920929
  'petal_width', '<=', 1.75
  'petal_length', '<=', 4.950000047683716
  'petal_width', '>', 1.6500000357627869
```

   <b>and the result is binary OR between all the paths for the given class</b>


In [5]:
import functools

def evaluate_tree(X, names, paths):
    def evaluate_expression(Xin, exp, names):
        v1 = Xin[ names.index(exp[0]) ]
        if exp[1] == "<=" :
            return int( exp[2] >= v1 )
        else:
            return int(v1 > exp[2])

            # reduce to all path
    return functools.reduce(lambda a1,b1: a1 + b1, # Arithmetic OR
        map(lambda expr: #reduce to a single tree path
                functools.reduce ( lambda a,b: a * b, # Arithmetic AND
                      map(lambda a: evaluate_expression( X, a, names), expr)),
        paths))

plain_result = [ evaluate_tree(X_test[i, :], feauters, path_tree)  for i in range(X_test.shape[0]) ]
plain_result

[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]

# Homorphic Implemention for Path evaluation

## Define operations

In [6]:
import requests
# Kuberantis TD Homomorphic encryption service
baseUrl = 'http://13.68.227.239/v1'

def encrypt(v):
    return requests.post(baseUrl + '/encrypt', params=dict(pubKey='alex', value=v)).json()['encrypted']
def decrypt(c):
    return requests.post(baseUrl + '/decrypt', params=dict(key='alex', value=c)).json()['value']
def mul(c1, c2):
    return requests.post(baseUrl + '/mul', params=dict(pubKey='alex', c1=c1, c2=c2)).json()['encrypted']
def add(c1, c2):
    return requests.post(baseUrl + '/add', params=dict(pubKey='alex', c1=c1, c2=c2)).json()['encrypted']
def compare(c1, c2):
    return requests.post(baseUrl + '/compare', params=dict(pubKey='alex', c1=c1, c2=c2)).json()['encrypted']

## Scaling to integers and encrypting testing data

<i>The other party (test data owner) is resposible for this operation</i>

In [7]:
scale = 10000
# scaling data
enc_vector_fn = lambda i: list(map(lambda a:  encrypt( int(round( a * scale )) ), X_test[i,:] ))
X_test_enc = [enc_vector_fn(i) for i in range(X_test.shape[0])]

## Scaling to integers and encrypting path thresholds
<i>the model owner is responsible for this operation</i>

In [8]:
#scaling path
path_tree_enc = list(
map(lambda a1: 
    list(map(lambda a2: [a2[0], a2[1], encrypt( int(round(a2[2] * scale))) ], a1 )), 
path_tree))

## Make Prediction by evauating the path list for Versicolor
 All data here is encrypted.<br>
 The computation is done my the model owner.<br>
 The model owner is not able to decrypt anything because he/she does not have the private key
 
 <b>This will be first the binary AND on the individual path, e.g</b>
    
 ```python   
  'petal_width', '>', encrypted(0.800000011920929)
  'petal_width', '<=', encrypted(1.75)
  'petal_length', '<=', encrypted(4.950000047683716)
  'petal_width', '>', encrypted(1.6500000357627869)
```

   <b>and the result is binary OR between all the paths for the given class</b>

In [9]:
def evaluate_tree_enc(X, names, paths): 
    def evaluate_expression_encrypted(Xin, exp, names):
        v1 = Xin[ names.index(exp[0]) ]
        if exp[1] == "<=" :
            return compare( exp[2] , v1 )
        else:
            return compare( v1 , exp[2]  )
    
    # reduce to all path
    return functools.reduce(lambda a1, b1: add(a1 , b1), # Arithmetic OR
        map(lambda expr: #reduce to a single tree path
                functools.reduce ( lambda a,b: mul(a ,  b), # Arithmetic AND
                      map(lambda a: evaluate_expression_encrypted( X, a, names), expr)),
        paths))

#decrypt(evaluate_tree_enc(X_test_enc[0], feauters, path_tree_enc))

## Evaluate final result
(!) note, result is still encrypted

In [10]:
enc_result = [ evaluate_tree_enc( X_test_enc[i], feauters, path_tree_enc)  for i in range(X_test.shape[0]) ]

## Data owner has a private key, so only he/she can decrypt the result

In [11]:
homomorphic_decrypted = list(map(lambda a: decrypt(a), enc_result))
homomorphic_decrypted

[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]

## Verify with non-encrypted computation

In [12]:
plain_result

[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]

##  Versicolor ( class 2 )  prediction from classifier itself

In [13]:
(clf.predict(X_test) == 2).astype(int).tolist()

[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]

# Conclusion 

Homomorphic encryption could be used in Decision tree and Random Forest to hide coefficients and data.
It will allow cryptographically separate a model and data.

This allows one organization to evaluate its model over hidden data of the other organization.
The computing organization does not see data and results. The other organization does not see the model structure and its coefficients.